# Loaders

> ...todo...

In [ ]:
#| default_exp loaders

In [ ]:
# | hide
import jupyter_black

jupyter_black.load()

In [ ]:
#| export
from sal.core import Data
from json import loads
from xml.etree import ElementTree

## ElementTree

To load from an ElementTree

In [ ]:
#| export
def element_tree_to_data(obj: ElementTree) -> Data:
    """Takes an ElementTree, and loads it into a Data instance"""
    attributes = obj.attrib.copy()
    data = Data(obj.tag, attributes)
    children = [element_tree_to_data(c) for c in obj]
    for c in children:
        data.add_child(c)
    return data

## xml

To load from xml

In [ ]:
#| export
def xml_to_data(xml: str) -> Data:
    """Takes a xml string, and loads it into a Data instance"""
    obj: ElementTree = ElementTree.fromstring(xml)
    return element_tree_to_data(obj)

In [ ]:
xml_to_data("<root at=\"b\"><child></child></root>")

<root {'at': 'b'}>
    <child {'at': 'b'} />
</root>

In [ ]:
#| export
def xml_file_to_data(path: str) -> Data:
    with open(path, 'r') as h:
        return xml_to_data(h.read())

In [ ]:
#| hide
assert xml_to_data("<root></root>") == Data('root')
assert xml_to_data("<root name=\"mauro\"></root>") == Data('root', {"name": "mauro"})

root = Data('root')
root.add_child(Data('child'))
assert xml_to_data("<root><child></child></root>") == root

## dict

To load from a dict

In [ ]:
#| export
def dict_to_data(arg: dict) -> Data:
    """Takes a dict, and loads it into a Data instance"""
    name = arg['name']
    del arg['name']
    
    children = arg.get('children', [])
    if children:
        del arg['children']
    
    data = Data(name, arg)
    for c in children:
        data.add_child(dict_to_data(c))
    return data

In [ ]:
dict_to_data({"name": "root", "children": [{"name": "child", "color": "blue"}]})

<root>
    <child {'color': 'blue'} />
</root>

In [ ]:
#| hide
assert dict_to_data({"name": "root"}) == Data('root')
assert dict_to_data({"name": "root", "age": 21}) == Data('root', {"age": 21})

root = Data('root')
root.add_child(Data('child'))
assert dict_to_data({"name": "root", "children": [{"name": "child"}]}) == root

## json

To load from json

In [ ]:
#| export
def json_to_data(arg:str) -> Data:
    """Takes a json string, and loads it into a Data instance"""
    j = loads(arg)
    return dict_to_data(j)

In [ ]:
json_to_data('{"name": "root", "children": [{"name": "child"}]}')

<root>
    <child />
</root>

In [ ]:
#| hide
json_st = '{"name": "root", "children": [{"name": "child"}]}'
root = Data('root')
root.add_child(Data('child'))
assert json_to_data(json_st) == root

---

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()